In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import html

#Test Scraper Code

In [ ]:
# Create a Beautiful Soup object
soup = BeautifulSoup(salepage.text, 'html.parser')



# Find all div elements with the specified class
divs = soup.find_all('div', class_='pull-left sc_listingTileContent')
count = 1
# Display the found divs
for div in divs:
    link_tag = div.find('a')
    if link_tag:
        propertyurl = "https://www.property24.co.ke" + link_tag['href']
        response = requests.get(property_url)
        property_soup = BeautifulSoup(response.text, 'html.parser')

        description_div = property_soup.find('div', class_='sc_listingDetailsText')
        if description_div:
          description = description_div.get_text().strip()
          print("Description:", description)
        else:
          description = "No description found for this property"
    else:
        propertyurl = None
    title = div.a['title']
    price_div = div.find('div', class_='sc_listingTilePrice')
    if price_div:
        price_spans = price_div.find_all('span')
        if len(price_spans) > 1:
            from_price = price_spans[1].text.strip()
            to_price = price_spans[3].text.strip()
            price = f"From {from_price} to {to_price}"
        else:
            price = price_spans[0].text.strip()
    else:
        price = None

    address = div.find('div', class_='sc_listingTileAddress').text.strip()
    teaser_div = div.find('div', class_='sc_listingTileTeaser')
    image_source = None
    if teaser_div.a and teaser_div.a.img:
        image_source = teaser_div.a.img['src']
    description = teaser_div.text.strip()  # Extract the entire description directly

    bed_img = div.find('img', class_='property24generic_icon_beds')
    baths_img = div.find('img', class_='property24generic_icon_baths')
    parking_img = div.find('img', class_='property24generic_icon_parking')

    beds = bed_img.find_next('span').get_text() if bed_img else ""
    baths = baths_img.find_next('span').get_text() if baths_img else ""
    parking = parking_img.find_next('span').get_text() if parking_img else ""

    # image_link = div.find('img', class_='pull-left')['src'] if div.find('img', class_='pull-left') else None

    print("Property URL:", propertyurl)
    print("Title:", title)
    print("Price:", price)
    print("Image Source:", image_source)
    print("Address:", address)
    print("Description:", description)
    print("Bedrooms:", beds)
    print("Bathrooms:", baths)
    print("Parking spaces:", parking)
    # print("Image Link:", image_link)
    imgs = div.find_all('img')
    for img in imgs:
        if 'src' in img.attrs:
            print("Image Link:", img['src'])
            break  # Break after finding the first image
    print("\n")



NameError: name 'salepage' is not defined

#Kenya Property24 For Sale

In [ ]:
import pandas as pd

for_sale_url = 'https://www.property24.co.ke/property-for-sale?provinceids=83,96,93,76,79,66,101,80,76,86&cityids=1890,1848,1847,1850,1868,1899,1901,2149'
salepage = requests.get(for_sale_url)
soup = BeautifulSoup(salepage.text, 'html.parser')

# Initialize an empty dataframe
data = {'Property URL': [], 'Title': [], 'Type': [], 'Price': [], 'Address': [], 'Description': [],
        'Bedrooms': [], 'Bathrooms': [], 'Parking spaces': [],
        'Floor Size': [], 'Erf Size': [], }
df = pd.DataFrame(data)

# Get data from all pages inside pagination
def getnextpage(soup):
  page = soup.find('div', {'class': 'text-center'})
  if not page.find('span', {'class': 'pull-right text-muted'}).find('a'):
    return  # If there's no span with the specified class containing an 'a' tag, end the search
  else:
    for_sale_url = page.find('span', {'class': 'pull-right text-muted'}).find('a')['href']
    return for_sale_url



while True:
  print(for_sale_url)
  for_sale_url = getnextpage(soup)
  if not for_sale_url:
    break
  salepage = requests.get(for_sale_url)
  soup = BeautifulSoup(salepage.text, 'html.parser')
  # Find all div elements with the specified class
  divs = soup.find_all('div', class_='pull-left sc_listingTileContent')
  count = 1
  # Display the found divs
  for div in divs:
      link_tag = div.find('a')
      if link_tag:
        propertyurl = "https://www.property24.co.ke" + link_tag['href']
        property_response = requests.get(propertyurl)
        property_soup = BeautifulSoup(property_response.text, 'html.parser')

        description_div = property_soup.find('div', class_='sc_listingDetailsText')
        if description_div:
          description = description_div.get_text().strip()

        else:
          description = "No description found for this property"
      else:
          propertyurl = None
      title = div.a['title']
      price_div = div.find('div', class_='sc_listingTilePrice')
      if price_div:
          price_spans = price_div.find_all('span')
          if len(price_spans) > 1:
              from_price = price_spans[1].text.strip()
              to_price = price_spans[3].text.strip()
              price = f"From {from_price} to {to_price}"
          else:
              price = price_spans[0].text.strip()
      else:
          price = None

      address = div.find('div', class_='sc_listingTileAddress').text.strip()
      bed_img = div.find('img', class_='property24generic_icon_beds')
      baths_img = div.find('img', class_='property24generic_icon_baths')
      parking_img = div.find('img', class_='property24generic_icon_parking')

      beds = bed_img.find_next('span').get_text() if bed_img else ""
      baths = baths_img.find_next('span').get_text() if baths_img else ""
      parking = parking_img.find_next('span').get_text() if parking_img else ""

      floor_unit = ""
      erf_unit = ""
      floor_size = ""
      erf_size = ""

      floor_size_span = div.find('span', class_='sc_erfSize')
      if floor_size_span:
          text = floor_size_span.get_text()
          if 'Floor Size' in text:
              floor_size = floor_size_span.find('span').get_text().strip()
              floor_unit = floor_size_span.contents[-1].strip()
          elif 'Erf Size' in text:
              erf_size = floor_size_span.find('span').get_text().strip()
              erf_unit = floor_size_span.contents[-1].strip()

      property_data = {
        'Property URL': propertyurl,
        'Title': title,
        'Type' : "For Sale",
        'Price': price,
        'Address': address,
        'Description': description,
        'Bedrooms': beds,
        'Bathrooms': baths,
        'Parking spaces': parking,
        'Floor Size': floor_size + ' ' + floor_unit,
        'Erf Size': erf_size + ' ' + erf_unit,
    }

      # Append the data to the DataFrame
      df = pd.concat([df, pd.DataFrame([property_data])], ignore_index=True)



In [ ]:
df.tail()


#Kenya Property 24 For Rent

In [ ]:
to_rent_url = 'https://www.property24.co.ke/property-to-rent?provinceids=83,96,93,76,79,66,101,80,76,86&cityids=1890,1848,1847,1850,1868,1899,1901,2149'
rentpage = requests.get(to_rent_url)
soup = BeautifulSoup(rentpage.text, 'html.parser')



# Get data from all pages inside pagination
def getnextpage(soup):
  page = soup.find('div', {'class': 'text-center'})
  if not page.find('span', {'class': 'pull-right text-muted'}).find('a'):
    return  # If there's no span with the specified class containing an 'a' tag, end the search
  else:
    to_rent_url = page.find('span', {'class': 'pull-right text-muted'}).find('a')['href']
    return to_rent_url

property_data_list = []

while True:
  print(to_rent_url)
  to_rent_url = getnextpage(soup)
  if not to_rent_url:
    break
  rentpage = requests.get(to_rent_url)
  soup = BeautifulSoup(rentpage.text, 'html.parser')
  # Find all div elements with the specified class
  divs = soup.find_all('div', class_='pull-left sc_listingTileContent')
  count = 1
  # Display the found divs
  for div in divs:
      link_tag = div.find('a')
      if link_tag:
        propertyurl = "https://www.property24.co.ke" + link_tag['href']
        property_response = requests.get(propertyurl)
        property_soup = BeautifulSoup(property_response.text, 'html.parser')

        description_div = property_soup.find('div', class_='sc_listingDetailsText')
        if description_div:
          description = description_div.get_text().strip()

        else:
          description = "No description found for this property"
      else:
          propertyurl = None
      title = div.a['title']
      # price_div = div.find('div', class_='sc_listingTilePrice')
      # if price_div:
      #     price_spans = price_div.find_all('span')
      #     if len(price_spans) > 1:
      #         from_price = price_spans[1].text.strip()
      #         to_price = price_spans[3].text.strip()
      #         price = f"From {from_price} to {to_price}"
      #     else:
      #         price = price_spans[0].text.strip()
      # else:
      #     price = None
      # Extract price and rent term
      price_div = div.find('div', class_='sc_listingTilePrice')
      if price_div:
          price_span = price_div.find('span')
          if price_span:
              price = price_span.text.strip()
          else:
              price = None

          rent_term_span = div.find('span', class_='sc_listingTilePriceRentTermDescription')
          if rent_term_span:
              rent_term = rent_term_span.text.strip()
          else:
              rent_term = None
      else:
          price = None
          rent_term = None


      address = div.find('div', class_='sc_listingTileAddress').text.strip()
      bed_img = div.find('img', class_='property24generic_icon_beds')
      baths_img = div.find('img', class_='property24generic_icon_baths')
      parking_img = div.find('img', class_='property24generic_icon_parking')

      beds = bed_img.find_next('span').get_text() if bed_img else ""
      baths = baths_img.find_next('span').get_text() if baths_img else ""
      parking = parking_img.find_next('span').get_text() if parking_img else ""

      floor_unit = ""
      erf_unit = ""
      floor_size = ""
      erf_size = ""

      floor_size_span = div.find('span', class_='sc_erfSize')
      if floor_size_span:
          text = floor_size_span.get_text()
          if 'Floor Size' in text:
              floor_size = floor_size_span.find('span').get_text().strip()
              floor_unit = floor_size_span.contents[-1].strip()
          elif 'Erf Size' in text:
              erf_size = floor_size_span.find('span').get_text().strip()
              erf_unit = floor_size_span.contents[-1].strip()

      property_data = {
        'Property URL': propertyurl,
        'Title': title,
        'Type' : "To Rent",
        'Price': price,
        'Payment Interval': rent_term,
        'Address': address,
        'Description': description,
        'Bedrooms': beds,
        'Bathrooms': baths,
        'Parking spaces': parking,
        'Floor Size': floor_size + ' ' + floor_unit,
        'Erf Size': erf_size + ' ' + erf_unit,
    }

      df = pd.concat([df, pd.DataFrame([property_data])], ignore_index=True)



In [ ]:
df.tail(30)

#Save CSV File of Property24 Kenya Properties

In [ ]:
from google.colab import files

df.to_csv('Property24Kenya.csv', index=False)
files.download('Property24Kenya.csv')

#Schedule to run the scraper every 10 hours


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Define your scraping functions
def scrape_data_for_sale(for_sale_url, df):
    # Your scraping logic for property for sale
    # ...

def scrape_data_for_rent(to_rent_url, df):
    # Your scraping logic for property to rent
    # ...

# Set initial for_sale_url and to_rent_url
for_sale_url = 'https://www.property24.co.ke/property-for-sale?provinceids=83,96,93,76,79,66,101,80,76,86&cityids=1890,1848,1847,1850,1868,1899,1901,2149'
to_rent_url = 'https://www.property24.co.ke/property-to-rent?provinceids=83,96,93,76,79,66,101,80,76,86&cityids=1890,1848,1847,1850,1868,1899,1901,2149'

# Initialize an empty dataframe
data = {'Property URL': [], 'Title': [], 'Type': [], 'Price': [], 'Address': [], 'Description': [],
        'Bedrooms': [], 'Bathrooms': [], 'Parking spaces': [],
        'Floor Size': [], 'Erf Size': [], }
df = pd.DataFrame(data)

# Set the interval (in seconds)
interval_seconds = 10 * 60 * 60  # 10 hours

# Run the script in a loop
while True:
    # Call your scraping functions
    scrape_data_for_sale(for_sale_url, df)
    scrape_data_for_rent(to_rent_url, df)

    # Print the dataframe
    print(df)

    # Save the dataframe to CSV
    df.to_csv('Property24Kenya.csv', index=False)

    # Wait for the specified interval before the next iteration
    time.sleep(interval_seconds)
